In [1]:
import pandas as pd
import numpy as np
from tqdm import notebook
import plotly.graph_objects as go
import plotly.express as px
import ast 

In [2]:
station = pd.read_csv('data/seoul_bike_station_01_12.csv',encoding="CP949",index_col=0)
near_bus = pd.read_csv('data/near_bus_200m.csv',encoding="CP949",index_col=0)
seoul_bike = pd.read_parquet('D:/git_local_repository/data_for_project/seoul_bike/seoul_bike_2021.parquet.gzip',columns=[ 'st_id1', 'st_id2', 'riding_time',
       'dist', 'm_pm'])

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return m

In [43]:
starting_bus_id = 13045
ending_bike_id = 754

# 출발, 도착
starting_point = near_bus[near_bus['id']==starting_bus_id]
ending_point = station.query('id==@ending_bike_id')

# 관련 노선 검색
bus_route_num = starting_point['bus'].tolist()
bus_route = near_bus.query('bus == @bus_route_num')

# 목적지 대여소와 가까운 버스 정류장 검색 -> list
result_km = haversine_np(ending_point['경도'].values, ending_point['위도'].values, bus_route['경도'].values, bus_route['위도'].values)

# 목적지 대여소 근방 1000m 이내 정류장 선택 -> df
bus_route.loc[:,'dist'] = result_km
bus_route_sorted = bus_route[bus_route['dist'] < 1500].dropna().drop_duplicates(subset=['id']).sort_values(by='id')


starting_point

c:\Users\679oo\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,bike_id,bus,order,id,name,경도,위도
453,"[180,198,3114]",N26,36,13045,충정로역,126.963492,37.561058
1717,"[180,198,3114]",N75,36,13045,충정로역,126.963492,37.561058


In [40]:
def bus_to_station(starting_bus_id: list[int] , ending_bike_id:int) :
    # ---- 버스 정류장 선택하기  ----

    # 출발, 도착
    starting_point = near_bus[near_bus['id']==starting_bus_id]
    ending_point = station.query('id==@ending_bike_id')

    # 관련 노선 검색
    bus_route_num = starting_point['bus'].tolist()
    bus_route = near_bus.query('bus == @bus_route_num')

    # 목적지 대여소와 가까운 버스 정류장 검색 -> list
    result_km = haversine_np(ending_point['경도'].values, ending_point['위도'].values, bus_route['경도'].values, bus_route['위도'].values)

    # 목적지 대여소 근방 1000m 이내 정류장 선택 -> df
    bus_route.loc[:,'dist'] = result_km
    bus_route_sorted = bus_route[bus_route['dist'] < 1500].dropna().drop_duplicates(subset=['id']).sort_values(by='id')


    # 검색 결과 없을때 검색 종료
    if bus_route_sorted.values.size == 0 : 
        print(bus_route_sorted)
        print('검색결과 없음')

    # 정류장과 목적지가 범위 내에 있는 경우 bus 추천 없이 바로 계산 --> df
    elif starting_bus_id in bus_route_sorted['id'].tolist() :
        bus_route_sorted = bus_route_sorted[bus_route_sorted['id'] == starting_bus_id]




    # ---- 버스 정류장 근방에 있는 대여소와 목적지 대여소 간 계산 ----

    # 추천_대여소_선정 : ast : '[123,]' => [123]으로 만들어줌 --> list
    b=[]
    for i in range(len(bus_route_sorted)) : 
        a = bus_route_sorted['bike_id'].iloc[i]
        a = ast.literal_eval(a)
        b.extend(a)
    starting_point_list = list(set(b))

    # 추천 대여소 리스트 만들기 
    BM_1 = (seoul_bike['st_id1'] == ending_point['id'].iloc[0]) & (seoul_bike['st_id2'].isin(starting_point_list))
    BM_2 = (seoul_bike['st_id2'] == ending_point['id'].iloc[0]) & (seoul_bike['st_id1'].isin(starting_point_list))
    seoul_bike_sorting = seoul_bike.loc[BM_1 | BM_2] # 거리 및 시간 계산을 위해 따로 추출

    # 버스정류장 근처 대여소와 집 근처 대여소 거리 비교
    ending_station = ending_point['id'].iloc[0]

    result_recomend =[]
    for starting_station in starting_point_list:
        BM_1 = (seoul_bike_sorting["st_id1"] == starting_station) & (seoul_bike_sorting["st_id2"] == ending_station)
        BM_2 = (seoul_bike_sorting["st_id2"] == starting_station) & (seoul_bike_sorting["st_id1"] == ending_station)

        # 평균거리 계산
        mean_station = (
            seoul_bike_sorting[BM_1 | BM_2]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)[:5]
            .index.values.mean()
        )

        # 이용기록 계산
        num_station = len(seoul_bike_sorting[BM_1 | BM_2])

        # 직선거리 계산
        lat_1 = station.query("id==@starting_station")["위도"].values
        lon_1 = station.query("id==@starting_station")["경도"].values
        lat_2 = station.query("id==@ending_station")["위도"].values
        lon_2 = station.query("id==@ending_station")["경도"].values
        dist_station = haversine_np(lat_1, lon_1, lat_2, lon_2)
        
        #자료 저장
        station_info = station.query('id==@starting_station')
        result_recomend.append([station_info['주소2'].iloc[0],station_info['위도'].iloc[0],station_info['경도'].iloc[0],starting_station, mean_station,num_station,f"{dist_station[0]/1000 :.2f}km"])

    #return 값    
    result_recomend = pd.DataFrame(result_recomend,columns=['주소2','위도','경도','id','time','num','dist'])
    bus_info = bus_route_sorted[['name','경도','위도','id','bus']]

    return result_recomend, bus_info, bus_route



# 16001 염창
# 13045 충정로
starting_bus_id = 16002
ending_bike_id = 754

result_recommend, bus_info = bus_to_station(starting_bus_id, ending_bike_id)




In [ ]:
# -- bus 정류장 그리기 --#
fig = px.scatter_mapbox(
    bus_info, lat="위도", lon="경도", hover_name="name", hover_data=["id"],zoom=11,height=500
)
fig.update_traces(
    marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.7, color="black")
)

# -- 추천대여소 대여소 그리기 -- #
fig_2 = px.scatter_mapbox(station.query("id == @starting_point_list"), lat="위도", lon="경도")
fig_2.update_traces(
    marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.7, color="blue")
)
fig.add_trace(fig_2.data[0])


# --반경 내 따릉이 대여소 그리기--#

# 거리 계산
station_haversine = haversine_np(
    ending_point["경도"].values,
    ending_point["위도"].values,
    station["경도"].values,
    station["위도"].values,
)

# 데이터 추가
around_station = station.copy()
around_station['dist'] = station_haversine
around_station = around_station[around_station['dist'] < 1000] 

# 시각화
fig_3 = go.Figure(
    go.Scattermapbox(
        name=f"station with in {round(1000/1000,2)}km",
        lat=around_station["위도"],
        lon=around_station["경도"],
        mode="markers",
        # marker = {'size': 20, 'symbol': "bus"},
        marker=go.scattermapbox.Marker(size=8, color="black", opacity=0.2),
        text=around_station["주소2"],
    )
)
fig.add_trace(fig_3.data[0])

# --목적지 대여소 그리기 -- #
fig_3 = go.Figure(
    go.Scattermapbox(
        name="location",
        lat=ending_point["위도"],
        lon=ending_point["경도"],
        mode="markers",
        # marker = {'size': 20, 'symbol': "bus"},
        marker=go.scattermapbox.Marker(size=10, color="fuchsia", opacity=0.8),
        text=ending_point["주소2"].values,
    )
)
fig.add_trace(fig_3.data[0])

fig.update_layout(
    mapbox=dict(
        accesstoken="pk.eyJ1IjoieWFuZ29vcyIsImEiOiJjbDNqd2tkN2IwbGdmM2pvNzF0c2M4NnZkIn0.J3IjPYg3w28cGiWkUD7bnA",
        # style='mapbox://styles/yangoos/cl3jubvl7000c14llgtoev0nm'
    ),
)
fig.show()


In [17]:
ending_point  = station.query("id == @ending_bike_id")[['경도','위도']].values[0]

starting_point = result_recommend[['경도','위도']].values[0]


126.874489

> ### 길찾기 기능(비추..)

In [32]:
import json
import urllib
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from tqdm import notebook
import warnings


# *-- Directions 5 활용 코드 --*
option = "traoptimal"
# option : 탐색옵션 [최대 3개, traoptimal(기본 옵션)
# / trafast, tracomfort, traavoidtoll, traavoidcaronly]


def get_optimal_route(start, goal, waypoints=["", ""], option=option):
    client_id = '44vlsxxinm'
    client_secret = 'BOHC6psFnLkjdRrlkLn24S1apLUTsuzrCnMuCi4A'
    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={start[0]},{start[1]}&waypoint={waypoints[0]},{waypoints[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    request.add_header("X-NCP-APIGW-API-KEY", client_secret)

    response = urllib.request.urlopen(request)
    res = response.getcode()

    if res == 200:
        response_body = response.read().decode("utf-8")
        return json.loads(response_body)

    else:
        print("ERROR")



starting_point = result_recommend[['경도','위도']].values[0]
ending_point  = station.query("id == @ending_bike_id")[['경도','위도']].values[0]
result = get_optimal_route(starting_point, ending_point)


# result.get('code')
a = result.get('route').get(option)[0].get('path')
b = pd.DataFrame(a)
b

fig = go.Figure(
    go.Scattermapbox(
        lat=b[1],
        lon=b[0],
        mode="lines",
        marker=dict(symbol="circle", size=15, color='#AD00FF' , opacity=0.5),
    )
)
fig.update_layout(
    mapbox=dict(
        accesstoken="pk.eyJ1IjoieWFuZ29vcyIsImEiOiJjbDNqd2tkN2IwbGdmM2pvNzF0c2M4NnZkIn0.J3IjPYg3w28cGiWkUD7bnA",
        center=go.layout.mapbox.Center(
            lat=37.58,
            lon=126.87
        ),
        # pitch=0,
        zoom=10
        # style='mapbox://styles/yangoos/cl3jubvl7000c14llgtoev0nm'
    ),
    margin=dict(l=20, r=20, t=20, b=20)
)
fig.show()




> ### 자전거 대여소 간 계산
* 해당 경우 검색 시 
* 지하철 -> 대여소 -> 대여소
* 대여소 -> 대여소 

In [ ]:


# 345ms
def st_to_st_new (starting_station, ending_station):

    lat_1=station.query('id==@starting_station')['위도'].values
    lon_1=station.query('id==@starting_station')['경도'].values
    lat_2=station.query('id==@ending_station')['위도'].values
    lon_2=station.query('id==@ending_station')['경도'].values
    dist_station = haversine_np(lat_1,lon_1,lat_2,lon_2)

    BM_1 = (seoul_bike['st_id1'] == starting_station) & (seoul_bike['st_id2'] == ending_station)
    BM_2 = (seoul_bike['st_id2'] == starting_station) & (seoul_bike['st_id1'] == ending_station)

    # 평균거리 계산
    sort_station = seoul_bike[BM_1 | BM_2]
    mean_station = sort_station[sort_station['dist'] < dist_station[0]*2]['riding_time'].value_counts()[:5].index.values.mean()
    num_station = len(sort_station)

    print(mean_station,num_station,f'{dist_station[0]/1000 :.2f}km' )

st_to_st_new(207, 754)


In [ ]:
## 옛날자료

def st_to_st(starting_station, ending_station):

    """
    starting_station = list or val이고 bike_station id가 들어감
    ending_station = list or val이고 bike_station id가 들어감
    """

    # int 또는 list로 starting station 받기
    if type(starting_station) == int:
        starting_station_list = list()
        starting_station_list.append(starting_station)
    else:
        starting_station_list = starting_station

    for starting_station in starting_station_list:
        BM_1 = (seoul_bike_sorting["st_id1"] == starting_station) & (
            seoul_bike_sorting["st_id2"] == ending_station
        )
        BM_2 = (seoul_bike_sorting["st_id2"] == starting_station) & (
            seoul_bike_sorting["st_id1"] == ending_station
        )

        # 평균거리 계산
        mean_station = (
            seoul_bike_sorting[BM_1 | BM_2]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)[:5]
            .index.values.mean()
        )

        # 이용기록 계산
        num_station = len(seoul_bike_sorting[BM_1 | BM_2])

        # 직선거리 계산
        lat_1 = station.query("id==@starting_station")["위도"].values
        lon_1 = station.query("id==@starting_station")["경도"].values
        lat_2 = station.query("id==@ending_station")["위도"].values
        lon_2 = station.query("id==@ending_station")["경도"].values
        dist_station = haversine_np(lat_1, lon_1, lat_2, lon_2)
        print(mean_station, num_station, f"{dist_station[0]/1000 :.2f}km")
    # return


st_to_st(counts_station, 754)
